In [ ]:
from collections import Counter
sorted([(name, Counter(name[0].lower().translate(dict.fromkeys(map(ord, 'aeiounrst .-'), None)))) for name in Card.objects.values_list('name')], key=lambda x: max(x[1].values() or [0]), reverse=True)


In [ ]:
from collections import Counter
Counter(''.join([name[0].lower() for name in Card.objects.values_list('name')]))